In [1]:
import eikon as ek
import eikon as tr
import pandas as pd
import time
ek.set_app_key('24851b66f5d640999f6bf3c0bef4bd71bc86c619')
#Das ist App Key (früher App ID) in Eikon Reuters von dem KIT account <- Search for App Key Generator in Eikon

AttributeError: module 'eikon' has no attribute 'set_app_key'

In [8]:
test_timeseries = ek.get_timeseries(["MSFT.O"], 
                       start_date="2016-01-01",  
                       end_date="2016-01-10")
print(test_timeseries)


MSFT.O         HIGH  CLOSE    LOW   OPEN     COUNT      VOLUME
Date                                                          
2016-01-04  54.8000  54.80  53.39  54.32  272781.0  53777963.0
2016-01-05  55.3900  55.05  54.54  54.93  180637.0  34079674.0
2016-01-06  54.4001  54.05  53.64  54.32  225856.0  39518863.0
2016-01-07  53.4850  52.17  52.07  52.70  303271.0  56564852.0
2016-01-08  53.2800  52.33  52.15  52.37  261949.0  48753969.0


In [16]:
test_close_timeseries = ek.get_timeseries(["MSFT.O"], fields="CLOSE", start_date="2017-03-01", end_date="2017-03-10", interval="daily", count=None, calendar="tradingdays", corax=None, normalize=False, raw_output=False, debug=False)
print(test_close_timeseries)

MSFT.O      CLOSE
Date             
2017-03-01  64.94
2017-03-02  64.01
2017-03-03  64.25
2017-03-06  64.27
2017-03-07  64.40
2017-03-08  64.99
2017-03-09  64.73
2017-03-10  64.93


In [20]:
close_timeseries_SP500=ek.get_timeseries([".SPX"], fields="CLOSE", start_date="2017-12-01", end_date="2018-01-10", interval="daily")
print(close_timeseries_SP500)

.SPX          CLOSE
Date               
2017-12-01  2642.22
2017-12-04  2639.44
2017-12-05  2629.57
2017-12-06  2629.27
2017-12-07  2636.98
2017-12-08  2651.50
2017-12-11  2659.99
2017-12-12  2664.11
2017-12-13  2662.85
2017-12-14  2652.01
2017-12-15  2675.81
2017-12-18  2690.16
2017-12-19  2681.47
2017-12-20  2679.25
2017-12-21  2684.57
2017-12-22  2683.34
2017-12-26  2680.50
2017-12-27  2682.62
2017-12-28  2687.54
2017-12-29  2673.61
2018-01-02  2695.81
2018-01-03  2713.06
2018-01-04  2723.99
2018-01-05  2743.15
2018-01-08  2747.71
2018-01-09  2751.29
2018-01-10  2748.23


In [119]:
#S&P 500 Joiners und Leavers Liste
f = ['TR.IndexJLConstituentChangeDate','TR.IndexJLConstituentName',
     'TR.IndexJLConstituentRIC','TR.IndexJLConstituentComName']
test_leavers = tr.get_data([".SPX"], fields=f, parameters={'IC':'L', 'SDate':'0D', 'EDate':'-24CY'})
test_leavers = test_leavers[0]
test_joiners = tr.get_data([".SPX"], fields=f, parameters={'IC':'J', 'SDate':'0D', 'EDate':'-24CY'})
test_joiners = test_joiners[0]

test_leavers['type'] = 'leaver'
test_joiners['type'] = 'joiner'
test_lj = pd.concat((test_joiners, test_leavers)).sort_values(by = ['Date'])

test_lj.to_csv("C:/Users/gp1396/Desktop/2018_12_20_SP500_leavers_joiners.csv", index = False)

In [120]:
#S&P 500 Constituents Liste
test_constituents = tr.get_data('.SPX', ['TR.IndexConstituentName','TR.IndexConstituentRIC'])[0]
test_constituents['type'] = 'constituent'
test_c = pd.DataFrame(test_constituents)
test_c.to_csv("C:/Users/gp1396/Desktop/2018_12_20_SP500_constituents.csv", index = False)
# in der Constituents Liste ist S&P500 selber dabei

In [137]:
#Eine Liste mit Joiners, Leavers, Constituents ohne Duplikate

test_ljc = pd.concat([test_lj, test_c], sort=False)
test_ljc = pd.DataFrame(test_ljc, columns = ['Instrument','Constituent Name','Constituent RIC'])
test_ljc = pd.DataFrame.drop_duplicates(test_ljc, subset='Constituent RIC')
test_ljc.to_csv("C:/Users/gp1396/Desktop/2018_12_20_SP500_ljc_without_duplicates.csv", index = False)

In [2]:
test_ljc = pd.read_csv("C:/Users/gp1396/Desktop/2018_12_20_SP500_ljc_without_duplicates.csv")
print(test_ljc.iloc[0][2])
print(test_ljc.iloc[1143][2])
n=test_ljc.shape[0]
print(n)

MDP.N
FTI.N
1144


In [3]:
n=test_ljc.shape[0]
for i in range(25, n):
    print(i, ': ', test_ljc.iloc[i][2])
    file_exist = False
    try:
        timeseries = ek.get_timeseries([test_ljc.iloc[i][2]], 
                           start_date="1990-01-01",  
                           end_date="1998-12-31")
        if timeseries.shape[0] > 0:
            timeseries.to_csv("C:/Users/gp1396/Desktop/stock prices historic/"+test_ljc.iloc[i][2]+".csv", index = True)
            file_exist = True
    except Exception as exc:
        if 'No data available for the requested date range' in str(exc):
            pass
        else:
            i -= 1
            continue
        
    time.sleep(1)
    try:
        timeseries = ek.get_timeseries([test_ljc.iloc[i][2]], 
                           start_date="1999-01-01",  
                           end_date="2008-12-31")
        if timeseries.shape[0] > 0:
            if file_exist:
                timeseries.to_csv("C:/Users/gp1396/Desktop/stock prices historic/"+test_ljc.iloc[i][2]+".csv", index = True, header = False, mode = 'a')
            else:
                timeseries.to_csv("C:/Users/gp1396/Desktop/stock prices historic/"+test_ljc.iloc[i][2]+".csv", index = True)
                file_exist = True
    except Exception as exc:
        if 'No data available for the requested date range' in str(exc):
            pass
        else:
            i -= 1
            continue
        
    time.sleep(1)
    try:
        timeseries = ek.get_timeseries([test_ljc.iloc[i][2]], 
                           start_date="2009-01-01",  
                           end_date="2018-12-20")
        if timeseries.shape[0] > 0:
            if file_exist:
                timeseries.to_csv("C:/Users/gp1396/Desktop/stock prices historic/"+test_ljc.iloc[i][2]+".csv", index = True, header = False, mode = 'a')
            else:
                timeseries.to_csv("C:/Users/gp1396/Desktop/stock prices historic/"+test_ljc.iloc[i][2]+".csv", index = True)
                file_exist = True
    except Exception as exc:
        if 'No data available for the requested date range' in str(exc):
            pass
        else:
            i -= 1
            continue
    time.sleep(1)

25 :  HAL.N
26 :  IFF.N
27 :  SPGI.N
28 :  AM^H13
29 :  FRE.N^G10
30 :  WOR
31 :  DAL.N^J05


Error with DAL.N^J05: No data available for the requested date range
DAL.N^J05: No data available for the requested date range | 


32 :  IPG.N
33 :  AAL.N^B97


Error with AAL.N^B97: No data available for the requested date range
AAL.N^B97: No data available for the requested date range | 
Error with AAL.N^B97: No data available for the requested date range
AAL.N^B97: No data available for the requested date range | 


34 :  PPG.N
35 :  UST.N^A09
36 :  X.N
37 :  COST.OQ
38 :  UCL.N^H05


...

Error with FANG.OQ: No data available for the requested date range
FANG.OQ: No data available for the requested date range | 
Error with FANG.OQ: No data available for the requested date range
FANG.OQ: No data available for the requested date range | 


1142 :  MXIM.OQ


Error with MXIM.OQ: No data available for the requested date range
MXIM.OQ: No data available for the requested date range | 
Error with FTI.N: No data available for the requested date range
FTI.N: No data available for the requested date range | 


1143 :  FTI.N


Error with FTI.N: No data available for the requested date range
FTI.N: No data available for the requested date range | 


In [179]:
    timeseries = ek.get_timeseries(['MDP'], 
                       start_date="2007-01-25",  
                       end_date="2018-12-20", count = 10000, interval = 'daily')
    timeseries.to_csv("C:/Users/gp1396/Desktop/MDP.csv", index = True, mode = 'a')

SyntaxError: positional argument follows keyword argument (<ipython-input-175-232820e7c65e>, line 1)